In [2]:
!pip install  openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.4 MB/s eta 0:00:0000:0100:01


In [3]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication


def sendmail(recipient_email, update, notebook_name, erorUpdate, attachment_path):

    sender_email = 'deepakpythonwork@gmail.com'
    sender_password = 'uzed ghqu tkxz mjxf '
    email_template_path = "index.html"
    with open(email_template_path, "r") as template_file:
        email_template = template_file.read()
    email_template = email_template.replace("{row number}", str(update))
    email_template = email_template.replace("{notebook}", str(notebook_name))
    email_template = email_template.replace("{bug}", str(erorUpdate))
    subject = 'Update ' + ' ' + str(notebook_name)  


    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = recipient_email
    message['Subject'] = subject

    message.attach(MIMEText(email_template, 'html'))
    with open(attachment_path, "rb") as attachment_file:
        attachment = MIMEApplication(attachment_file.read(), _subtype="xlsx")
        attachment.add_header('Content-Disposition', 'attachment', filename=attachment_path)
        message.attach(attachment)

    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()  
        server.login(sender_email, sender_password)  
        server.sendmail(sender_email, recipient_email, message.as_string()) 

In [ ]:
sendmail("rohit45deepak@gmail.com", "Ram issue", "UBS", "data") # test message

In [4]:
import requests, json, os
import pandas as pd
from openpyxl import load_workbook
def getdata(start_date, end_date, rows, page):
    cookies = {
        'BIGipServervds2-app.prod.us.issapps.com_8330': '2483286538.35360.0000',
        'BIGipServervds.issgovernance.com_8120': '889319946.47135.0000',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-GB,en;q=0.5',
        # 'Accept-Encoding': 'gzip, deflate, br',
        'X-Requested-With': 'XMLHttpRequest',
        'Connection': 'keep-alive',
        'Referer': 'https://vds.issgovernance.com/vds/',
        # 'Cookie': 'BIGipServervds2-app.prod.us.issapps.com_8330=2483286538.35360.0000; BIGipServervds.issgovernance.com_8120=889319946.47135.0000',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
    }

    params = {
        'customerID': 'MjU0',
        'fromDate': start_date,
        'toDate': end_date,
        'liveSiteYN': '1',
        'random': '0.49761613755807876',
        'locale': 'en',
        'MeetingTypeList': '',
        'CountryList': '',
        'VotedList': '',
        '_search': 'false',
        'nd': '1705577300279',
        'rows': rows,
        'page': page,
        'SortByColumn': 'CompanyName',
        'OrderBy': 'asc',
    }

    response = requests.get('https://vds.issgovernance.com/vds/api/getVdsData/14', params=params, headers=headers)
    return response.json()['data']



def getCompanydata(MultipleFundIDs, MeetingId):
    cookies = {
        'BIGipServervds2-app.prod.us.issapps.com_8330': '2483286538.35360.0000',
        'BIGipServervds.issgovernance.com_8120': '889319946.47135.0000',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-GB,en;q=0.5',
        # 'Accept-Encoding': 'gzip, deflate, br',
        'X-Requested-With': 'XMLHttpRequest',
        'Connection': 'keep-alive',
        'Referer': 'https://vds.issgovernance.com/vds/',
        # 'Cookie': 'BIGipServervds2-app.prod.us.issapps.com_8330=2483286538.35360.0000; BIGipServervds.issgovernance.com_8120=889319946.47135.0000',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
    }

    params = {
        'customerID': 'MjU0',
        'fromDate': '',
        'toDate': '',
        'fundValue': MultipleFundIDs,
        'liveSiteYN': '1',
        'meetingID': MeetingId,
        'random': '0.956757217578498',
        'locale': 'en',
        '_search': 'false',
        'nd': '1705577418341',
        'rows': '2000',
        'page': '1',
        'SortByColumn': 'Proposal',
        'OrderBy': 'asc',
    }

    response = requests.get('https://vds.issgovernance.com/vds/api/getVdsData/7', params=params, headers=headers)

    return response.json()['data']
def getTotalrow(start_date, end_date, rows, page):
     TRow= getdata(start_date, end_date, rows, page)[0]['TotalRows']
     return TRow

# Optimize memory

In [5]:
start_date = '2023-01-01'
end_date = '2023-12-31'
#processed  2925 of 18655 rows
start_row= 2925 
file_name= 'UBS.xlsx'
rows = getTotalrow(start_date, end_date, 20, 1)
print(f"Total rows {rows}")
rowProcessed=0
data = getdata(start_date, end_date, rows, 1)
try:
    for i in range(start_row, len(data)):
        dataDict = {"CompanyName": data[i]['CompanyName'], "Ticker": data[i]['Ticker'], "Country": data[i]['Country'], "SecurityID": data[i]['SecurityID'], "MeetingDate": data[i]['MeetingDate'], "MeetingType": data[i]['MeetingType'], "FundName": data[i]['FundName'], "MultipleFundIDs": data[i]['MultipleFundIDs'], "MeetingID": data[i]['MeetingID'] }
        companyData = getCompanydata(dataDict['MultipleFundIDs'], dataDict['MeetingID'])
        DataSheet = pd.DataFrame(companyData)
        if os.path.exists(file_name):
            rowProcessed=i
            df = pd.read_excel(file_name)
            # df = df.append(DataSheet, ignore_index= True)
            df = pd.concat([df, DataSheet], ignore_index=True)
            df.to_excel(file_name, index=False)
            print(f"processed  {i + 1} of {rows} rows" )
        else:
            DataSheet.to_excel(file_name, index=False)
    sendmail("rohit45deepak@gmail.com", rowProcessed, "UBS", "Done file check", file_name)
    sendmail("eleanorpwilli@gmail.com", rowProcessed, "UBS", "Done file check", file_name)
except Exception as e:
    sendmail("rohit45deepak@gmail.com", rowProcessed, "UBS", e, file_name)
    # sendmail("eleanorpwilli@gmail.com", rowProcessed, "UBS", e)


Total rows 18655
processed  2926 of 18655 rows
processed  2927 of 18655 rows
processed  2928 of 18655 rows
processed  2929 of 18655 rows
processed  2930 of 18655 rows
processed  2931 of 18655 rows
processed  2932 of 18655 rows
processed  2933 of 18655 rows
processed  2934 of 18655 rows
processed  2935 of 18655 rows
processed  2936 of 18655 rows
processed  2937 of 18655 rows
processed  2938 of 18655 rows
processed  2939 of 18655 rows
processed  2940 of 18655 rows
processed  2941 of 18655 rows
processed  2942 of 18655 rows
processed  2943 of 18655 rows
processed  2944 of 18655 rows
processed  2945 of 18655 rows
processed  2946 of 18655 rows
processed  2947 of 18655 rows
processed  2948 of 18655 rows
processed  2949 of 18655 rows
processed  2950 of 18655 rows
processed  2951 of 18655 rows
processed  2952 of 18655 rows
processed  2953 of 18655 rows
processed  2954 of 18655 rows
processed  2955 of 18655 rows
processed  2956 of 18655 rows
processed  2957 of 18655 rows
processed  2958 of 1865

KeyboardInterrupt: 

In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'
rows = getTotalrow(start_date, end_date, 20, 1)
print(f"Total rows {rows}")

data = getdata(start_date, end_date, rows, 1)
ResultData = []
for i in data:
    dataDict = {"CompanyName": i['CompanyName'], "Ticker": i['Ticker'], "Country": i['Country'], "SecurityID": i['SecurityID'], "MeetingDate": i['MeetingDate'], "MeetingType": i['MeetingType'], "FundName": i['FundName'], "MultipleFundIDs": i['MultipleFundIDs'], "MeetingID": i['MeetingID'] }
    print(f"processed  {data.index(i) + 1} of {rows} rows" )
    companyData = getCompanydata(dataDict['MultipleFundIDs'], dataDict['MeetingID'])
    ResultData.extend(companyData)
    DataSheet = pd.DataFrame(ResultData)
    DataSheet.to_excel("UBS.xlsx")

processed  1301 of 18655 rows
processed  1302 of 18655 rows
processed  1303 of 18655 rows
processed  1304 of 18655 rows
processed  1305 of 18655 rows
processed  1306 of 18655 rows
processed  1307 of 18655 rows
processed  1308 of 18655 rows
processed  1309 of 18655 rows
processed  1310 of 18655 rows
processed  1311 of 18655 rows
processed  1312 of 18655 rows
processed  1313 of 18655 rows
processed  1314 of 18655 rows
processed  1315 of 18655 rows
processed  1316 of 18655 rows
processed  1317 of 18655 rows
processed  1318 of 18655 rows
processed  1319 of 18655 rows
processed  1320 of 18655 rows
processed  1321 of 18655 rows
processed  1322 of 18655 rows
processed  1323 of 18655 rows
processed  1324 of 18655 rows
processed  1325 of 18655 rows
processed  1326 of 18655 rows
processed  1327 of 18655 rows
processed  1328 of 18655 rows
processed  1329 of 18655 rows
processed  1330 of 18655 rows
processed  1331 of 18655 rows
processed  1332 of 18655 rows
processed  1333 of 18655 rows
processed 

KeyboardInterrupt: 

<img src="image-20240122-153741.png" width="" align="" />

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=851a2575-1bd9-47a6-8e3b-3f39909b983b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>